In [1]:
import pandas as pd
import numpy as np

In [2]:
# load data
def load_data(filename):
    df = pd.read_csv(filename)
    return df

In [3]:
# convert timestamp
def convert_timestamp(timestamp):
    timestamp = str(timestamp)
    hour = timestamp[8:10]
    minute = timestamp[10:12]
    second = timestamp[12:14]
    milliseconds = timestamp[14:]
    time = int(hour) * 36000000 + int(minute) * 600000 + int(second) * 10000 + int(milliseconds)
    
    return time

In [4]:
# clean data and get cleaned dataframe
def clean_data(df):
    df['timeIndex'] = df['timestamp'].apply(convert_timestamp)
    base = df['timeIndex'][0]
    df['relativeTime'] = df['timeIndex'] - base
    df['second'] = df['relativeTime'] // 10000
    df_cleaned = df.groupby(['second']).mean()
    del df_cleaned['timestamp']
    del df_cleaned['timeIndex']
    del df_cleaned['relativeTime']
    
    return df_cleaned

In [5]:
# preprocess data
def preprocess_data(filename):
    df = load_data(filename)
    df_cleaned = clean_data(df)
    
    return df_cleaned

In [6]:
import scipy.stats

actionList = []
fireList = []
for i in range(1, 17):
    action = '/Users/cwang/USF/HCI_Final_ML/action/' + str(i) + '.csv'
    opponent = '/Users/cwang/USF/HCI_Final_ML/opponent/' + str(i) + '.csv'
    df_action = preprocess_data(action)
    df_opponent = preprocess_data(opponent)
    df_opponent.columns = ['opponent_up', 'opponent_down', 'opponent_left', 'opponent_right', 'opponent_fire']
    df = pd.concat([df_action, df_opponent], axis=1, join_axes=[df_action.index])
    up = scipy.stats.pearsonr(df['up'], df['opponent_up'])
    df['action'] = df['up'] + df['down'] + df['left'] + df['right']
    df['opponent_action'] = df['opponent_up'] + df['opponent_down'] + df['opponent_left'] + df['opponent_right']
    actionCorr = scipy.stats.pearsonr(df['action'], df['opponent_action'])
    actionList.append(actionCorr)
    fireCorr = scipy.stats.pearsonr(df['fire'], df['opponent_fire'])
    fireList.append(fireCorr)

/Users/cwang/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


### Pearson Correlation

In [7]:
df_actionCorr = pd.DataFrame(actionList)

In [8]:
df_fireCorr = pd.DataFrame(fireList)

In [9]:
df_fireCorr.describe()

,0,1
count,15.000000,1.600000e+01
mean,0.294740,1.738332e-01
std,0.183634,2.829752e-01
min,-0.043331,9.057886e-07
25%,0.153175,1.684259e-04
50%,0.291536,6.558116e-02
75%,0.437830,1.395034e-01
max,0.600387,1.000000e+00


In [10]:
df_actionCorr

,0,1
0,0.859032,1.121692e-14
1,0.849749,9.768146e-20
2,0.154037,4.828393e-01
3,0.777512,6.020635e-27
4,0.973082,2.984539e-32
5,0.838541,1.361806e-27
6,0.506514,7.577940e-07
7,0.470783,9.447707e-09
8,0.353864,1.981656e-03
9,0.583546,1.331913e-14
